# Excercise 08: Release Strategies


## AB Testing

Reset the previous deployment by executing `kubectl delete seldondeployments <yourname-model>`.

Create a new file in the folder `pipeline` named `ab-seldon-model.yml`:

```yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: <yourname>-model
spec:
  name: <yourname>-model
  predictors:
    - componentSpecs:
        - spec:
            containers:
            - name: dummy-classifier
              image: fluescher/sustaino-dummy-service
      graph:
        children: []
        endpoint:
          type: REST
        name: dummy-classifier
        type: MODEL
      name: dummy-classifier
      replicas: 1
      traffic: 20
    - componentSpecs:
        - spec:
            containers:
            - name: real-classifier
              image: gcr.io/academy-300812/<yourname>/eligiblity-service:latest
      graph:
        children: []
        endpoint:
          type: REST
        name: real-classifier
        type: MODEL
      name: real-classifier
      replicas: 1
      traffic: 80
```

Apply this configuration: `kubectl apply -f pipeline/ab-seldon-model.yml`

Verify that the pods are started correctly by issueing `kubectl get pods`. Are all up and running?


You can generate load: `NAMESPACE=<yourname> MODEL=<yourname>-model ../services/create-load.sh`


Replace `<yourname>` with your name.

Questions:
- Generate some load to this model. Can you verify if both deployments get traffic? Check Prometheus
- How is the user impacted by this deployment?



## Shadowing

To test a shadow deployment, first remove the previous deployment using `kubectl delete -f pipeline/ab-seldon-model.yml`:


Create a new file in the folder `pipelines` named `shadow-seldon-model.yml`:


```yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: <yourname>-model
spec:
  name: <yourname>-model
  predictors:
    - componentSpecs:
        - spec:
            containers:
            - name: dummy-classifier
              image: fluescher/sustaino-dummy-service
      graph:
        children: []
        endpoint:
          type: REST
        name: dummy-classifier
        type: MODEL
      name: dummy-classifier
      replicas: 1
    - componentSpecs:
        - spec:
            containers:
            - name: real-classifier
              image: gcr.io/academy-300812/<yourname>/eligiblity-service:latest
      graph:
        children: []
        endpoint:
          type: REST
        name: real-classifier
        type: MODEL
      name: real-classifier
      replicas: 1
      shadow: true
      traffic: 100
```

Apply this configuration: `kubectl apply -f pipeline/shadow-seldon-model.yml`

Verify that the pods are started correctly by issueing `kubectl get pods`. Are all up and running?

You can generate load: `NAMESPACE=<yourname> MODEL=<yourname>-model ../services/create-load.sh`


Questions:
- Generate some load to this model. Can you verify if both deployments get traffic? Check Prometheus
- How is the user impacted by this deployment?